# LAB1: Dynamic Programming

This lab will ask you to implement the following dynamic programming algorithms:

* **Iterative Value Policy Evaluation**: to evaluate a given policy knowing the environment model.
* **Policy Iteration**: to find the optimal policy for a given environment model.
* **Value Iteration**: another approach to find the optimal policy for a given environment model.

## Map Environment Overview

To test the algorithms you need to implement you will need a *stochastic* environment from which we new its state transition model. 

The `MapEnv` class provides a customizable environment for Reinforcement Learning (RL) algorithms, specifically designed to simulate a grid-like world where an agent can navigate. This environment is structured similarly to the Gymnasium interface, making it compatible with various RL techniques. The key features of this environment include state representation, action management, and a stochastic option for simulating randomness in actions.

The map structure is a square grid where the agent can move in four directions: down, up, right, and left. The agent's goal is to reach a specific location on the grid while avoiding obstacles, that are randomly placed in the environment. Free space is indicated as `0` and obstacles as `1` in the `map`attribute. The goal state is stored in the `goal_id` attibute (`goal` attribute also contains the goal but as a (x, y) tuple instead of a single value).

The following methods are available in the `MapEnv` class:

### 1. **Initialization (`__init__`)**
- **Purpose**: Initializes the environment with a specified size, obstacle density, and stochastic behavior.
- **Parameters**:
  - `size`: The dimension of the square grid (default is 4).
  - `obstacles_percent`: The proportion of obstacles to generate in the environment (default is 0.5).
  - `stochastic`: A boolean indicating if the actions should be stochastic (default is `False`).

### 2. **Plotting the Environment (`plot`)**
- **Purpose**: Visualizes the current state of the environment, including the agent's position, the goal, and obstacles.
- **Output**: A printed representation of the grid with symbols indicating the agent's current position ('O'), the goal ('★'), and obstacles ('■').

### 3. **State Representation (`state`)**
- **Purpose**: Returns a unique integer representing the current position of the agent in the grid.
- **Output**: An integer that maps the agent's coordinates to a single state value.

### 4. **Action Execution (`action`)**
- **Purpose**: Updates the agent's position based on the selected action while calculating the associated reward.
- **Parameters**: 
  - `a`: The action to be performed, represented as an integer. Available actions are: `down: 0`, `up: 1`, `right: 2`, and `left: 3`.
- **Output**: A tuple containing the reward for the action and a boolean indicating if the goal has been reached.

### 5. **Stochastic Action Execution (`sthocastic_action`)**
- **Purpose**: Simulates a stochastic action where there is a probability of taking a random action instead of the intended one. In this environment the probability of taking the intended action is 0.85, and the probability of taking any of the other 3 actions is 0.15.
- **Parameters**:
  - `a`: The intended action.
- **Output**: The result of executing the action, accounting for stochastic behavior.

### 6. **Model Dynamics (`model`)**
- **Purpose**: Provides a model of the environment, returning possible next states, rewards, and probabilities of transitioning from the current state given a specific action.
- **Parameters**:
  - `state`: The current state of the agent.
  - `action`: The action the agents wants to take.
- **Output**: A tuple containing lists of next states, rewards, and transition probabilities.

### 7. **Taking a Step (`step`)**
- **Purpose**: Executes a given action and returns the current state, reward, and whether the goal has been reached. Follows a similar interface than the [Gymnasium](https://gymnasium.farama.org/index.html) environments.
- **Parameters**: 
  - `a`: The action to be executed.
- **Output**: A tuple consisting of the new state, the reward, and a boolean indicating completion.

### 8. **Resetting the Environment (`reset`)**
- **Purpose**: Resets the agent's position to the starting state.
- **Output**: The initial state of the environment.

### 9. **Check if state is valid (`is_valid`)**
- **Purpose**: Check is state `s` is free or contains an obstacle.
- **Parameters**:
  - `s`: The state to be checked.
- **Output**: `True` if the state is free, `False` if it contains an obstacle.


You can know the number of states and actions of the environment by accessing the `n_states` and `n_actions` attributes of the environment object.

## Exercise 1: Get familiarized with the given environment

In this exercise, you will get familiarized with the `MapEnv` environment. You will create an instance of the environment and interact with it to understand its functionalities. You will also visualize the environment to see the agent's position, the goal, and the obstacles.
Try to creeate an instance of the environment with different parameters and observe the changes in the environment. Once experienced, define the environment with the following parameters:
- `size`: 4
- `obstacles_percent`: 0.4
- `stochastic`: True

Be sure that there is a path from the agent initial position (state = 0 which belongs to cell (0, 0)) to the goal indicated as a star (★) using only the four actions: down, up, right, and left. Once all the algorithms works for the 4x4 environment, you can try with bigger environments (10x10, 20x20, etc.) to see how the algorithms scale.

In [137]:
from map_env import MapEnv
import numpy as np

env = MapEnv(size=4, obstacles_percent=0.4, stochastic=True)
env.plot()

[['O' '■' '★' ' ']
 [' ' ' ' ' ' ' ']
 ['■' ' ' '■' '■']
 [' ' ' ' ' ' ' ']] 



## Exercise 2.1: Implement the value iteration algorithm

Implementing the value iteration algorithm, defined as follows:

**Iterative Value Policy Evaluation Algorithm**

1. **Initialize** $V(s) = 0 $ for all $ s \in S $.
2. **Repeat**:
   - Set $\Delta \leftarrow 0 $.
   - For each $s \in S $:
     - Set $v \leftarrow V(s)$.
     - Update $V(s)$ to the maximum of the sum:
       $
       V(s) \leftarrow \max_a \sum_{s'} P_{ss'}^{\pi(s)} [R(s') + \gamma V(s')]
       $
     - Update $ \Delta $ to the maximum of $ \Delta $ and the absolute difference:
       $
       \Delta \leftarrow \max(\Delta, |v - V(s)|)
       $
3. **Until** $ \Delta < \theta $ (a small threshold).

In [184]:
def iterative_value_policy_evaluation(env: object, policy: np.ndarray, gamma: float=0.9, theta: float=0.1) -> np.ndarray:
    V = np.zeros(shape=(env.n_states,))
    while True:
        delta = 0
        for i in range(env.n_states):
            v = V[i]
            if env.is_valid(i) == False:
                V[i] = -np.inf
                continue
            else:
                next_states, rewards, probability_trans = env.model(i, policy[i])
                V_temp = 0
                for j in range(len(next_states)):
                    V_temp += probability_trans[j] * (rewards[j] + gamma * V[next_states[j]])
                V[i] = V_temp
                delta = np.maximum(delta, np.abs(v - V[i]))

        if delta < theta:
            break

    return V

## Exercise 2.2: Test the iterative_value_policy_evaluation function

Create a random policy for the generated environment and use the implemented `iterative_value_policy_evaluation` algorithm to find out the value function $V$. You can visualize the policy and the value function $V$ using the auxiliar functions defined in `rl_auc.py` file.

In [185]:
from rl_aux import print_policy

# Create a random policy
policy = np.random.randint(0, env.n_actions, size=(env.n_states,))
print_policy(env, policy)

[['↑' '■' '★' '←']
 ['←' '↑' '↑' '↑']
 ['■' '←' '■' '■']
 ['←' '↑' '↑' '↑']] 



In [ ]:
from rl_aux import print_value_function

# Compute and plot the value function
V = iterative_value_policy_evaluation(env, policy, gamma=0.9, theta=0.1)
print_value_function(env, V)

[['-8.0' '  ■ ' '  ★ ' '-3.7']
 ['-5.3' '-7.6' '-3.5' '-4.4']
 ['  ■ ' '-9.1' '  ■ ' '  ■ ']
 ['-8.2' '-9.1' '-7.9' '-8.0']] 



## Exercise 3.1: 
Implement the Policy Iteration algorithm defined as follows:

**Policy Iteration Algorithm**

1. **Initialize** $ \pi, \forall s \in S $ to a random action $ a \in A(s) $, arbitrarily.
2. **Repeat**:
   - Set $ \pi' \leftarrow \pi $.
   - Compute $ V^{\pi} $ for all states using a policy evaluation method.
   - For each state $ s $:
     - Update $ \pi(s) $ to:
       $\pi(s) \leftarrow \arg \max_{a \in A} \sum_{s'} P_{ss'}^{a} [R(s') + \gamma V^{\pi}(s')]$
3. **Until** $ \pi(s) = \pi'(s) \, \forall s $.

In [187]:
# Policy iteration
def policy_iteration(env:object, gamma: float=0.9, theta: float=0.1):
    policy = np.random.randint(0, env.n_actions, size=(env.n_states,))
    while True:
        policy_prime = policy.copy()
        V = iterative_value_policy_evaluation(env, policy, gamma=gamma, theta=theta)
        for i in range(env.n_states):
            if env.is_valid(i) == False:
                continue
            else:
                V_max = -np.inf
                for j in range(env.n_actions):
                    V_temp = 0
                    next_states, rewards, probability_trans = env.model(i, j)
                    for k in range(len(next_states)):
                        V_temp += probability_trans[k]*(rewards[k] + gamma*V[next_states[k]])
                    if(V_max < V_temp):
                        V_max = V_temp
                        policy[i] = j

        if (policy == policy_prime).all() == True:
            break

    return policy, V

## Exercise 3.2: Test the policy_iteration function

Use the implemented `policy_iteration` algorithm to find the optimal policy for the generated environment. You can visualize the policy and the value function $V$ using the auxiliar functions defined in `rl_aux.py` file.

In [188]:
# Test policy iteration algorithm
policy, V = policy_iteration(env, gamma=0.9, theta=0.1)
print_policy(env, policy)
print_value_function(env, V)

[['←' '■' '★' '←']
 ['←' '➜' '↑' '←']
 ['■' '↑' '■' '■']
 ['←' '➜' '↓' '←']] 

[['-2.1' '  ■ ' '  ★ ' '-1.1']
 ['-2.9' '-2.1' '-1.1' '-2.1']
 ['  ■ ' '-3.0' '  ■ ' '  ■ ']
 ['-2.9' '-2.1' '-1.1' '-2.1']] 



## Exercise 4.1: Implement the Value Iteration algorithm

Implement the Value Iteration algorithm defined as follows:

**Iterative Value Function Evaluation Algorithm**

1. **Initialize** $ V(s) $ for all $ s \in S $ arbitrarily (for instance to 0).
2. **Repeat**:
   - Set $ \Delta \leftarrow 0 $.
   - For each $ s \in S $:
     - Set $ v \leftarrow V(s) $.
     - Update $ V(s) $ to:
       $
       V(s) \leftarrow \max_{a} \sum_{s'} P_{ss'}^{a} [R(s') + \gamma V(s')]
       $
     - Update $ \pi(s) $ to:
       $
       \pi(s) \leftarrow \arg \max_{a \in A} \sum_{s'} P_{ss'}^{a} [R(s') + \gamma V^{\pi}(s')]
       $
     - Update $ \Delta $ to:
       $
       \Delta \leftarrow \max(\Delta, |v - V(s)|)
       $
3. **Until** $ \Delta < \theta $ (a small threshold).
4. **Return** deterministic policy, $ \pi $, such that:
   $
   \pi(s) = \arg \max_{a \in A} \sum_{s'} P_{ss'}^{a} [R(s') + \gamma V^{\pi}(s')]
   $

In [193]:
# Value iteration
def value_iteration(env:object, gamma: float=0.9, theta: float=0.0001):
    V = np.zeros(shape=(env.n_states,))
    while True:
        delta = 0
        for i in range(env.n_states):
            v = V[i]
            if env.is_valid(i) == False:
                V[i] = -np.inf
                continue
            V_max = -np.inf
            for j in range(env.n_actions):
                V_temp = 0
                next_states, rewards, probability_trans = env.model(i, j)
                for k in range(len(next_states)):
                    V_temp += probability_trans[k]*(rewards[k] + gamma*V[next_states[k]])
                
                if V_max < V_temp:
                    V_max = V_temp
            
            V[i] = V_max
            V_state = -np.inf
            for j in range(env.n_actions):
                V_temp = 0
                next_states, rewards, probability_trans = env.model(i, j)
                for k in range(len(next_states)):
                    V_temp += probability_trans[k]*(rewards[k] + gamma*V[next_states[k]])
                
                if V_state < V_temp:
                    V_state = V_temp
                    policy[i] = j

            delta = np.maximum(delta, np.abs(v - V[i]))  
        if delta < theta:
            break

    return policy, V

## Exercise 4.2: Test the value_iteration function

Use the implemented `value_iteration` algorithm to find the optimal policy for the generated environment. You can visualize the policy and the value function $V$ using the auxiliar functions defined in `rl_aux.py` file.

In [195]:
# Test value iteration algorithm
policy, V = value_iteration(env, gamma=0.9, theta=0.1)
print_policy(env, policy)
print_value_function(env, V)

[['←' '■' '★' '←']
 ['←' '➜' '↑' '←']
 ['■' '↓' '■' '■']
 ['←' '➜' '↓' '←']] 

[['-2.1' '  ■ ' '  ★ ' '-1.1']
 ['-2.9' '-2.1' '-1.1' '-2.1']
 ['  ■ ' '-2.9' '  ■ ' '  ■ ']
 ['-2.9' '-2.1' '-1.1' '-2.1']] 

